In [1]:
# Install libraries to google colab environment
!pip install hazm
!pip install stopwords_guilannlp

    100% |████████████████████████████████| 317kB 12.1MB/s 
    100% |████████████████████████████████| 1.4MB 15.5MB/s 
    100% |████████████████████████████████| 235kB 27.5MB/s 
  Running setup.py bdist_wheel for nltk ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Running setup.py bdist_wheel for libwapiti ... - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
# Import libraries
import json
import nltk
import numpy as np
# from __future__ import unicode_literals
from hazm import *
import pandas as pd
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import collections
import re
from collections import Counter, defaultdict
from stopwords_guilannlp import stopwords_output

In [3]:
# Upload dataset to google colab environment
from google.colab import files
uploaded = files.upload()

Saving out.jsonl to out.jsonl


In [83]:
file = open('out.jsonl', 'r')
s = file.readlines()


print('There is {} item available'.format(len(s)))

There is 1000 item available


##  Json example:
   

```
{
  "_id": "5b4f7280020eb20597f40200",
  "NewsAgency": "AsrIran",
  "newsCode": "621582",
  "newsLink": "http://www.asriran.com/fa/news/621582",
  "date": "تاریخ انتشار:  ۱۴:۴۷ - ۲۷ تير ۱۳۹۷ - 18 July 2018",
  "newsPath": "صفحه نخست » اقتصادی",
  "newsPathLinks": {
    "صفحه نخست": "/fa/archive?service_id=1",
    "اقتصادی": "/fa/archive?service_id=1&amp;cat_id=4"
  },
  "title": "تعداد شعب داخلی شبکه بانکي کاهش یافت",
  "rutitr": "",
  "subtitle": "تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به 20598 شعبه ‌رسيد.",
  "body": "تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397 نسبت به پايان اسفند ماه&nbsp; 1396، 0.6 درصد کاهش یافت.به گزارش عصرایران به نقل از بانک مرکزی، تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به&nbsp; 20598&nbsp; شعبه &zwnj;رسيد.بر این اساس و در پایان خرداد ماه 1397، تعداد شعب بانک هاي دولتي 8681 شعبه ( 42.1 درصد) ، بانک هاي خصوصي شده 6461 شعبه ( 31.4 درصد) و بانک هاي خصوصي و موسسات اعتباري 5456 شعبه ( 26.5 درصد) است.همچنین تعداد کل شعب ريالي ـ ارزي شبکه بانکي کشور بالغ بر 3787 شعبه، تعداد شعب شهري 19735 و شعب روستايي 863 شعبه است.بر اساس این گزارش تعداد شعب خارج از کشور شبکه بانکي تعداد 48 شعبه و تعداد باجه هاي بانکي کشور در پايان خرداد ماه 1397 تعداد 2274&nbsp; باجه است و213 شعبه بانکی نیز در مناطق آزاد تجاري قرار دارند.بررسی تعداد شعب داخل کشور بانک ها و موسسات اعتباری&nbsp; به تفکیک استانی در پایان خرداد ماه سال 1397 نیز نشان می دهد که استان&zwnj;هاي تهران، اصفــهان و خراسان رضوي به &zwnj;ترتيب با 4049، 1431 و 1318 شعبه داراي بيشترين تعداد شعب هستند.گزارش فصلی &laquo;اطلاعات تعداد شعب بانک ها&nbsp; و موسسات اعتباری&raquo; توسط اداره اطلاعات بانکی بانک مرکزی تهیه و در آن تعداد شعب&nbsp; بانک ها و موسسات اعتباری به تفکیک گروه های بانکی و استانی اعلام می شود.",
  "bodyHtml": " <div align=\"justify\"><img class=\"image_btn\" style=\"margin: 0px 14px;\" title=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" src=\"http://cdn.asriran.com/files/fa/news/1397/4/27/875553_322.jpg\" alt=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" width=\"250\" height=\"169\" align=\"left\"/>تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397 نسبت به پايان اسفند ماه&nbsp; 1396، 0.6 درصد کاهش یافت.<br/><br/>به گزارش <span style=\"color: #0000cd;\">عصرایران</span> به نقل از بانک مرکزی، تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به&nbsp; 20598&nbsp; شعبه &zwnj;رسيد.<br/><br/>بر این اساس و در پایان خرداد ماه 1397، تعداد شعب بانک هاي دولتي 8681 شعبه ( 42.1 درصد) ، بانک هاي خصوصي شده 6461 شعبه ( 31.4 درصد) و بانک هاي خصوصي و موسسات اعتباري 5456 شعبه ( 26.5 درصد) است.<br/><br/>همچنین تعداد کل شعب ريالي ـ ارزي شبکه بانکي کشور بالغ بر 3787 شعبه، تعداد شعب شهري 19735 و شعب روستايي 863 شعبه است.<br/><br/><img class=\"image_btn\" style=\"margin: 0px auto; display: block;\" title=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" src=\"http://cdn.asriran.com/files/fa/news/1397/4/27/875556_701.jpg\" alt=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" width=\"404\" height=\"300\" align=\"\"/><br/><br/>بر اساس این گزارش تعداد شعب خارج از کشور شبکه بانکي تعداد 48 شعبه و تعداد باجه هاي بانکي کشور در پايان خرداد ماه 1397 تعداد 2274&nbsp; باجه است و213 شعبه بانکی نیز در مناطق آزاد تجاري قرار دارند.<br/><br/>بررسی تعداد شعب داخل کشور بانک ها و موسسات اعتباری&nbsp; به تفکیک استانی در پایان خرداد ماه سال 1397 نیز نشان می دهد که استان&zwnj;هاي تهران، اصفــهان و خراسان رضوي به &zwnj;ترتيب با 4049، 1431 و 1318 شعبه داراي بيشترين تعداد شعب هستند.<br/><br/>گزارش فصلی &laquo;اطلاعات تعداد شعب بانک ها&nbsp; و موسسات اعتباری&raquo; توسط اداره اطلاعات بانکی بانک مرکزی تهیه و در آن تعداد شعب&nbsp; بانک ها و موسسات اعتباری به تفکیک گروه های بانکی و استانی اعلام می شود.</div> <div class=\"wrapper\"/> ",
  "tags": {
    "بانک مرکزی": "/fa/tag/1/بانک مرکزی",
    "شعب داخلی": "/fa/tag/1/شعب داخلی"
  }
}

```



## Load jsons into Dataframe

In [84]:
def prepare_json(j):
  # clear newsPath
  newsPath = j['newsPath']
  newsPath = newsPath.replace('عصرايران دو » ','')
  newsPath = newsPath.replace('صفحه نخست » ','')
  j['newsPath'] = newsPath
  return j
    
        
raw_data = []
for item in s:
  try:
    j = json.loads(item)
    prepare_json(j)
    # print(list(j['tags'].keys())[0])
    raw_data.append({'title': j['title'], 'body':j['body'], 'newsAgency': j['NewsAgency'], 'newsPath':j['newsPath'], 'tag':list(j['tags'].keys())[0] })
  except:
    pass
df = pd.DataFrame(raw_data)
df.head()

,body,newsAgency,newsPath,tag,title
0,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,AsrIran,ورزشی,استقلال,افتخاری قید ماندن در هیات مدیره استقلال را هم زد
1,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,AsrIran,اجتماعی,مالیات,دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...
2,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,AsrIran,اقتصادی,اوپک,قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...
3,رئیس فراکسیون فرهنگیان مجلس خطاب به وزیر آموزش...,AsrIran,اجتماعی,آموزش و پرورش,حاجی‌بابایی به بطحایی:آقای وزیر در اطلاع‌رسانی...
4,رئیس صندوق بین&zwnj;المللی پول در آستانه نشست ...,AsrIran,بین الملل,صندوق بین المللی پول,رئیس صندوق بین‌المللی پول: آمریکا از جنگ تعرفه...


# Encoding categorical features 

In [85]:
categories = ['newsAgency','newsPath']

le = LabelEncoder()
df[categories] = df[categories].apply(le.fit_transform)

# give the content we need
x = df[['title','body','tag']]
# split categories of every post from newsPath
y_category = df['newsPath']
# split agency of every post
y_agency = df ['newsAgency']

x.head()

,title,body,tag
0,افتخاری قید ماندن در هیات مدیره استقلال را هم زد,مدیرعامل سابق استقلال از عضویت در هیات مدیره ا...,استقلال
1,دادستان انتظامی مالیاتی سازمان مالیات: آخرین ا...,دادستان انتظامی مالیاتی سازمان امور مالیاتی گف...,مالیات
2,قیمت سبد نفتی اوپک یک گام دیگر عقب نشست/ 70 دل...,قیمت سبد نفتی اوپک دیروز به روند کاهشی خود ادا...,اوپک
3,حاجی‌بابایی به بطحایی:آقای وزیر در اطلاع‌رسانی...,رئیس فراکسیون فرهنگیان مجلس خطاب به وزیر آموزش...,آموزش و پرورش
4,رئیس صندوق بین‌المللی پول: آمریکا از جنگ تعرفه...,رئیس صندوق بین&zwnj;المللی پول در آستانه نشست ...,صندوق بین المللی پول


# Split TrainSet and TestSet

In [86]:
# Split for category predictor
x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(x, y_category, test_size=0.3, random_state=42)
print('--Category Predictor--')
print('Train size: ',len(x_cat_train))
print('Test size : ',len(x_cat_test))

# Split for category predictor
x_ag_train, x_ag_test, y_ag_train, y_ag_test = train_test_split(x, y_agency, test_size=0.3, random_state=42)
print('--Agency Predictor--')
print('Train size: ',len(x_ag_train))
print('Test size : ',len(x_ag_test))

--Category Predictor--
Train size:  698
Test size :  300
--Agency Predictor--
Train size:  698
Test size :  300


# Preprocess data

In [0]:
stop_set = stopwords_output("Persian", "set")
customized_stop_words = ['&zwnj;','zwnj;','&zwnj', '&laquo;', '&raquo;', '&nbsp;', 'ViewAjax', 'asran', 'lse', 'pletely', 'secd', 'vent', 'ventDeult', 'ad_type', 'ads', 'advert', 'adverts', 'ajax', 'also', 'api', 'bull', 'cdn', 'click', 'clip', 'com', 'completely', 'data', 'disable', 'document', 'embed', 'enable', 'fa', 'false', 'feature', 'files', 'find', 'flowplayer', 'for', 'fp', 'function', 'height', 'hellip', 'http', 'if', 'ima', 'ir', 'keyboard', 'laquo', 'ldquo', 'lrm', 'max_ad_duration', 'mp4', 'nbsp', 'ndash', 'news', 'on', 'player_', 'playlist', 'post', 'pre', 'prevent', 'preventDefault', 'raquo', 'ratio', 'rdquo', 'ready', 'return', 'rlm', 'roll', 'sdmax', 'second', 'seeking', 'set', 'sources', 'splash', 'src', 'standardvideo', 'this', 'time', 'true', 'type', 'ui', 'up', 'var', 'via', 'video', 'videoViewAjax', 'width', 'www', 'zwnj']
# Remove some redundant words before tokenization and vectorization

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False
  
def sentence_preprocess(sentence):
  tokenized = word_tokenize(sentence)
  tokens = []
  for word in tokenized:
    # Remove digits
    # Remove 1 letter words
    for sw in customized_stop_words:
      word = word.replace(sw, '')
    if not (word.isdigit() or isfloat(word) or len(word)<2 or (word in stop_set)):
      tokens.append(word)
  # Return as a string
  tokens = ' '.join(tokens)
  return tokens

def preprocess(data):
  data_list = []
  for sentence in data:
    data_list.append(sentence_preprocess(sentence))
  return data_list

# Vectorizer (Count Vectorzer | TF-IDF Vectorizer)

In [204]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1))
# vectorizer = CountVectorizer(lowercase=False, ngram_range=(1,1))
# For category predictor
# train set
x_cat_train_corpus = x_cat_train['body'].values
x_cat_train_corpus = preprocess(x_cat_train_corpus)
# test set
x_cat_test_corpus = x_cat_test['body'].values
x_cat_test_corpus = preprocess(x_cat_test_corpus)

# For agency predictor
# train set
# x_ag_train_corpus = x_ag_train['title'].values
# x_ag_train_corpus = preprocess(x_ag_train_corpus)
# test set
# x_ag_test_corpus = x_ag_test['title'].values
# x_ag_test_corpus = preprocess(x_ag_test_corpus)

# Fit vectorizer
vectorizer.fit(x_cat_train_corpus)
# vectorizer.fit(x_ag_train_corpus)

# Print vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)
print(len(vocab))

# Transform vectorizer over train and test set
x_cat_train_vec = vectorizer.transform(x_cat_train_corpus).toarray()
x_cat_test_vec = vectorizer.transform(x_cat_test_corpus).toarray()
# x_ag_train_vec = vectorizer.transform(x_ag_train_corpus).toarray()
# x_ag_test_vec = vectorizer.transform(x_ag_test_corpus).toarray()

['00', '0W', '0به', '11', '12', '13', '1397', '14', '15', '16', '18', '19', '1990', '1994', '1996', '1998', '1XXX', '2002', '2006', '2010', '2014', '21', '23', '24', '25', '26', '27', '29', '2XXX', '2گل', '30', '39', '3بازی', '3برابر', '40', '44', '45', '49', '4میلیونی', '54', '56', '5W', '5لیگ', '60', '6s', '6با', '7ماهه', '7هزار', '873923', '873970', '873974', '873990', '874015', '874027', '874092', '874172', '874297', '874368', '874381', '874394', '874450', '874553', '874664', '874668', '874693', '874824', '874829', '874843', '874852', '874882', '874983', '875182', '875220', '875238', '875244', '875250', '875257', '875260', '875361', '875414', '875481', '875515', '875518', '875528', '875604', '8تماس', '8هزار', '91', '92', 'A2', 'AB', 'AFC', 'ATR', 'Aare', 'Abnb', 'Adidas', 'Amanotes', 'Android', 'Ar', 'Ari', 'August', 'Aways', 'B1', 'BB', 'Bar', 'Ba۱', 'Ba۲', 'Beautul', 'Bernard', 'Big', 'Black', 'Blancco', 'Blanco', 'Brain', 'Bunis', 'C3', 'CAF', 'CDN', 'CKD', 'CMAJ', 'COC', 'CON',

In [205]:
print('Shape of x_train vector for category predictor: ', x_cat_train_vec.shape)
print('Shape of x_test vector for category predictor: ', x_cat_test_vec.shape)
# print('Shape of x_train vector for agency predictor: ', x_ag_train_vec.shape)
# print('Shape of x_test vector for agency predictor: ', x_ag_test_vec.shape)

Shape of x_train vector for category predictor:  (698, 19247)
Shape of x_test vector for category predictor:  (300, 19247)


In [0]:
# Print the first vector (sparse matrix should be convert to array)
# If you want to print the full array, just uncomment following line
# np.set_printoptions(threshold=np.nan)
# Default configuration
# np.set_printoptions(threshold=1000)
# print(x_cat_train_vec[100])

# Naive Bayes Implementation

In [0]:
# Prior probabilities for the classes
def class_probability(data):
  example_numb = len(data)
  prob = dict(Counter(data))
  for key in prob.keys():
    prob[key] = prob[key] / float(example_numb)
  return prob

In [0]:
def feature_probability(data, sum_of_words, V):
  # Calculating the sum of all weights (Count in CountVectorizer or TF-IDF value in TfidfVectorzier)
  sum_of_weights = 0.0
  for w in data:
    sum_of_weights += w
  # Calculating probabilty of this feature, using Add 1 Smoothing 
  p = (sum_of_weights + 1) / (sum_of_words + V)
  return p

In [0]:
def naive_bayes_train(x, y, voc):
  # x = x_train_vec, y = y_train, voc = vocab
  
  # Calculating probability of each class
  y_probability = class_probability(y)
  
  # List of all unique class
  y_classes = np.unique(y)
  print ('This model have %d class' % len(y_classes))
  
  # Shape of input
  x_rows, x_columns = x.shape
  print ('This model data have %d rows and %d columns' % (x_rows, x_columns))
  
  # Probabilites of each feature based on each class
  likelihoods = {}

  # Initialize
  for cls in y_classes:
    likelihoods[cls] = defaultdict(list)

  for cls in y_classes:
    # For every class
    # Find which doument category (y_train) is same as current class
    row_indices = np.where(y == cls)[0]
    
    # Get the documents (sentences vector)
    docs = x[row_indices, :]
    
    # Calculating all word weights
    sum_of_words = 0
    for d in docs:
      for v in d:
        sum_of_words += v
        
    # Remember the shape of this class (number of document and the length of vectors)
    doc_numb, vec_len  = np.shape(docs)
    
    # For every feature (word), add it's value to current class
    for i in range(0, vec_len):
      likelihoods[cls][i] += list(docs[:,i])
      
    # Calculate probablity of every feature given this class
    for i in range(0, vec_len):
      likelihoods[cls][i] = feature_probability(likelihoods[cls][i], sum_of_words, len(voc))
      
  return y_probability, likelihoods
  

In [210]:
p_class, p_features = naive_bayes_train(x_cat_train_vec, y_cat_train, vocab)
# p_class is probability of each class
# print(p_class)
# p_features consist of probability of each feature based on each class
# print(p_features)

This model have 17 class
This model data have 698 rows and 19247 columns


In [0]:
def naive_bayes_test(x, y, class_prob, features_prob):
  
  prediction = {}
  y_list = list(y)
  
  # For every test data
  for i in range(0, len(x)):
    current_x = x[i]
    current_y = y_list[i]
    
    # A dictionary that contains prediction value based on each class
    current_prediction = {}
    for cls in class_prob.items():
      
      predict = 1
      
      # Multiply by probability of current class
      current_cls_index = cls[0]
      current_cls_value = cls[1]
      predict = predict * current_cls_value
      
      # For each feature (word) get the probability based on current class
      for index, word in enumerate(current_x):
        if word > 0 :
          word_probability = (features_prob[current_cls_index][index] ** word)
          predict = predict * word_probability
          
      # Add new predict to prediction list of this test data
      current_prediction[current_cls_index] = predict
      
    # Sort prediction
    sorted_by_value = sorted(current_prediction.items(), key=lambda kv: kv[1], reverse=True)
    
    # Add prediction list of this test data to global predictation list
    prediction[i] = sorted_by_value
    
  return prediction

In [0]:
def get_labels_of_prediction(pred, numb):
  result = {}
  for index, data in enumerate(pred.items()):
    labels = []
    for l in data[1]:
      labels.append(l[0])
    if numb == 'All':
      result[index] = labels
    else:
      result[index] = labels[0:numb]
  return result

In [213]:
# Make prediction
prediction = naive_bayes_test(x_cat_test_vec, y_cat_test, p_class, p_features)
print(prediction)

# Get the top N class (here is 1)
prediction_labels = get_labels_of_prediction(prediction, 1)
print(prediction_labels)

# Convert to a single value list
y_cat_predicted = []
for p in prediction_labels.items():
  y_cat_predicted.append(p[1][0])
print(y_cat_predicted)

# Convert real y_test to a list
y_cat_real = list(y_cat_test)
print(y_cat_real)

{0: [(1, 7.701318059434048e-45), (11, 1.4585863520671745e-45), (0, 1.3672368044281354e-45), (2, 4.207151534629827e-46), (16, 3.609721353966712e-46), (4, 9.036662500940748e-47), (10, 5.553166742690032e-47), (14, 3.695487220644665e-47), (3, 3.4219912705107515e-47), (9, 1.2229258723377567e-47), (13, 1.1753083864975692e-47), (15, 8.846815710379365e-48), (8, 7.238051130805914e-48), (12, 6.240625038918227e-48), (6, 4.5476818601843356e-48), (7, 4.4434143314927614e-48), (5, 2.9202364426203053e-48)], 1: [(1, 5.84792973469556e-32), (0, 1.3791611064324567e-32), (11, 9.562384204995057e-33), (16, 4.070243045457918e-33), (2, 3.5781190772166134e-33), (4, 1.3792160141892285e-33), (10, 5.3826251596744995e-34), (14, 4.924765040089951e-34), (3, 4.700410207151784e-34), (9, 1.7256782124798853e-34), (13, 1.5668907500083181e-34), (15, 1.2247154960510591e-34), (8, 9.83863385755155e-35), (12, 8.600834765726932e-35), (7, 6.213179183390743e-35), (6, 5.978834291103704e-35), (5, 3.912468893794846e-35)], 2: [(16, 6

In [214]:
# Convert list to numpy array
a = np.array(y_cat_predicted)
b = np.array(y_cat_real)

# Find correct items
correct = (a == b)

# Calculating Accuracy
accuracy = correct.sum() / correct.size

print('Accurarcy is :' , accuracy*100)

Accurarcy is : 57.333333333333336


# Test

In [0]:
c_vec = CountVectorizer(lowercase=False, ngram_range=(1,1))
test_c_vec = ['این یک مثال است', 'این یک مثال دیگر است', 'این مثال دیگری است']
c_vec.fit(test_c_vec)
vocab = c_vec.get_feature_names()
print(vocab)
print(len(vocab))
vect = c_vec.transform(test_c_vec).toarray()
print (vect)
test_y = (1, 2, 2)
class_prob = occurrences(test_y)
print (class_prob)

test_classes = np.unique(test_y)

test_likelihoods = {}
for cls in test_classes:
  test_likelihoods[cls] = defaultdict(list)

for cls in test_classes:
  # For every class
  # Find which doument category (y_train) is same as current class
  row_indices = np.where(test_y == cls)[0]
  # Get the documents (sentences vector)
  docs = vect[row_indices, :]
  sum_of_words = 0
  for d in docs:
    for v in d:
      sum_of_words += v
  # Remember the shape of this class (number of document and the length of vectors)
  doc_numb, vec_len  = np.shape(docs)
  # For every feature (word), add it's value to current class
  for i in range(0, vec_len):
    test_likelihoods[cls][i] += list(docs[:,i])
  for i in range(0, vec_len):
    test_likelihoods[cls][i] = feature_probability(test_likelihoods[cls][i], sum_of_words, len(vocab))

print(test_likelihoods)
test_x = ['این دیگر مثال است']
test_x = c_vec.transform(test_x).toarray()
test_x = test_x[0]
test_y = 2
current_x = test_x
current_y = test_y

# A dictionary that contains prediction value based on each class
current_prediction = {}
for cls in class_prob.items():
  predict = 1
  # Multiply by probability of current class
  current_cls_index = cls[0]
  current_cls_value = cls[1]
  predict = predict * current_cls_value
  print('predict multply with class', predict, 'class',current_cls_index)
  # For each feature (word) get the probability based on current class
  for index, word in enumerate(current_x):
    if word > 0 :
      print(index, word, test_likelihoods[current_cls_index][index])
      word_probability = (test_likelihoods[current_cls_index][index] ** word)
      predict = predict * word_probability
      print('p' , predict)
  print('predict', predict)
  current_prediction[current_cls_index] = predict
print (current_prediction)

['است', 'این', 'دیگر', 'دیگری', 'مثال', 'یک']
6
[[1 1 0 0 1 1]
 [1 1 1 0 1 1]
 [1 1 0 1 1 0]]
{1: 0.3333333333333333, 2: 0.6666666666666666}
{1: defaultdict(<class 'list'>, {0: 0.2, 1: 0.2, 2: 0.1, 3: 0.1, 4: 0.2, 5: 0.2}), 2: defaultdict(<class 'list'>, {0: 0.2, 1: 0.2, 2: 0.13333333333333333, 3: 0.13333333333333333, 4: 0.2, 5: 0.13333333333333333})}
predict multply with class 0.3333333333333333 class 1
0 1 0.2
p 0.06666666666666667
1 1 0.2
p 0.013333333333333334
2 1 0.1
p 0.0013333333333333335
4 1 0.2
p 0.00026666666666666673
predict 0.00026666666666666673
predict multply with class 0.6666666666666666 class 2
0 1 0.2
p 0.13333333333333333
1 1 0.2
p 0.02666666666666667
2 1 0.13333333333333333
p 0.0035555555555555557
4 1 0.2
p 0.0007111111111111111
predict 0.0007111111111111111
{1: 0.00026666666666666673, 2: 0.0007111111111111111}
